In [3]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'selenium'

In [2]:
def draftkings_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    odds = [x.text for x in soup.findAll('span',{'class', input_dict['odds_class']})]
    # odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::3], outcomes[2::3], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

def fanduel_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

def bovada_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[2::7], odds[3::7], odds[4::7], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

In [3]:
prem_dir = {'draftkings':{'url':'https://sportsbook.draftkings.com/leagues/soccer/53591936',
                          'parser': draftkings_parse,
                          'args':{'outcomes_class':'sportsbook-outcome-cell__label',
                                  'odds_class':'sportsbook-odds american default-color'}},
            'fanduel':{'url':'https://sportsbook.fanduel.com/sports/navigation/730.1/7567.1',
                       'parser': fanduel_parse,
                       'args':{'outcomes_class':'name',
                               'odds_class':'selectionprice'}},
            'bovada':{'url':'https://www.bovada.lv/sports/soccer/england-premier-league',
                      'parser': bovada_parse,
                      'args':{'outcomes_class':'name',
                              'odds_class':'bet-price'}}
           }
           

In [5]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = 'driver/chromedriver'
column_names = ['home_team','away_team', 'home_odds','tie_odds', 'away_odds', 'source', 'datetime']

for key in prem_dir:    
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get(prem_dir[key]['url'])
    driver.implicitly_wait(3)

    data = prem_dir[key]['parser'](prem_dir[key]['args'],driver,key)
    
    ### Needs to add data persistenceb
    
    print(pd.DataFrame(data, columns=column_names).head())
    
    driver.quit()

           home_team         away_team home_odds tie_odds away_odds  \
0     Leicester City          Brighton      -148     +285      +410   
1          Tottenham          West Ham      -155     +310      +410   
2  Manchester United  Sheffield United      -240     +335      +750   
3          Newcastle       Aston Villa      +130     +230      +220   
4             Wolves       Bournemouth      -182     +320      +510   

       source                   datetime  
0  draftkings 2020-06-22 17:51:08.995727  
1  draftkings 2020-06-22 17:51:08.995727  
2  draftkings 2020-06-22 17:51:08.995727  
3  draftkings 2020-06-22 17:51:08.995727  
4  draftkings 2020-06-22 17:51:08.995727  
   home_team    away_team home_odds tie_odds away_odds   source  \
0  Leicester     Brighton      -145     +270      +440  fanduel   
1  Tottenham     West Ham      -155     +290      +420  fanduel   
2  Newcastle  Aston Villa      +135     +230      +210  fanduel   
3     Wolves  Bournemouth      -180     +310   

In [6]:
matches_538 = pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv')

In [10]:
prem_matches = matches_538[matches_538['league'] == 'Barclays Premier League']

In [14]:
columns = ['date', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2']

In [20]:
prem_matches[columns].sort_values('date', ascending=False)

,date,league,team1,team2,spi1,spi2,prob1,prob2,probtie,proj_score1,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
10137,2020-07-26,Barclays Premier League,Chelsea,Wolverhampton,85.43,81.95,0.4641,0.2655,0.2704,1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10136,2020-07-26,Barclays Premier League,Burnley,Brighton and Hove Albion,69.85,71.45,0.3753,0.3391,0.2856,1.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,2020-07-26,Barclays Premier League,Leicester City,Manchester United,81.32,88.01,0.2967,0.4338,0.2695,1.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10132,2020-07-26,Barclays Premier League,Everton,AFC Bournemouth,78.42,63.65,0.5600,0.1969,0.2432,1.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10131,2020-07-26,Barclays Premier League,Crystal Palace,Tottenham Hotspur,69.70,76.94,0.2988,0.4325,0.2687,1.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,2019-08-10,Barclays Premier League,Watford,Brighton and Hove Albion,67.64,62.74,0.5133,0.2371,0.2496,1.67,...,31.3,38.8,0.0,3.0,0.73,0.74,1.68,1.01,0.00,2.99
640,2019-08-10,Barclays Premier League,West Ham United,Manchester City,69.97,94.24,0.0997,0.7393,0.1610,0.82,...,23.4,76.1,0.0,5.0,1.27,2.99,0.51,1.84,0.00,4.20
685,2019-08-10,Barclays Premier League,Burnley,Southampton,67.63,68.57,0.4450,0.2956,0.2594,1.53,...,32.8,26.0,3.0,0.0,0.97,1.16,1.12,1.28,3.04,0.00
696,2019-08-10,Barclays Premier League,Tottenham Hotspur,Aston Villa,85.07,64.64,0.7290,0.0906,0.1804,2.18,...,61.8,32.3,3.0,1.0,3.09,0.69,2.41,0.46,2.64,1.05


In [21]:
prem_matches[prem_matches['date']=='2020-06-24']

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
8986,2019,2020-06-24,2411,Barclays Premier League,Wolverhampton,AFC Bournemouth,81.59,63.35,0.6032,0.1639,...,78.6,66.9,1.0,0.0,0.75,0.10,0.69,0.32,1.05,0.00
8987,2019,2020-06-24,2411,Barclays Premier League,Newcastle,Aston Villa,64.41,60.95,0.3669,0.3670,...,1.3,53.3,1.0,1.0,0.71,1.33,0.79,1.86,1.05,1.05
8989,2019,2020-06-24,2411,Barclays Premier League,Manchester United,Sheffield United,86.71,71.24,0.6739,0.1061,...,98.5,27.8,3.0,0.0,3.00,0.12,2.87,0.31,3.06,0.00
8990,2019,2020-06-24,2411,Barclays Premier League,Norwich City,Everton,59.83,78.01,0.2238,0.5230,...,12.8,3.1,0.0,1.0,0.46,0.73,0.74,1.31,0.00,1.05
9012,2019,2020-06-24,2411,Barclays Premier League,Liverpool,Crystal Palace,92.03,71.25,0.7330,0.0893,...,0.4,7.9,4.0,0.0,2.92,0.10,2.23,0.17,4.20,0.00
